In [1]:
import requests
import re
import hashlib
import json
import time
import hashlib
import random
from pyquery import PyQuery as pq

In [2]:
data=[]

In [3]:
url = 'https://www.instagram.com/explore/tags/%E5%8F%B0%E5%8D%97/'
uri= 'https://www.instagram.com/graphql/query/?query_hash=f92f56d47dc7a55b606908374b43a314&variables={user_id}'

In [4]:
headers = {
'Connection':'keep-alive',
'Host':'www.instagram.com',
'Referer':'https://www.instagram.com/explore/tags/%E6%88%90%E5%A4%A7/',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
'X-Requested-With':'XMLHttpRequest'
}

In [5]:
def hashStr(strInfo):
    h = hashlib.md5()
    h.update(strInfo.encode("utf-8"))
    return h.hexdigest()

In [6]:
def get_urls(url):
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        else:    
            print('请求错误状态码：', response.status_code)        
    except Exception as e:
        print(e)
        return None

In [11]:
def get_json(url,headers):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            return response.json()
        else:
            print('请求网页json错误, 错误状态码：', response.status_code)
    except Exception as e:
        print(e)
        time.sleep(60 + float(random.randint(1, 4000))/100)
        return get_json(url)

In [30]:
def crawl(html):
    j=0
    user_id=''
    profile = {}
    doc = pq(html)
    items = doc('script[type="text/javascript"]').items()
    
    #print(user_id )
    GIS_rhx_gis = re.findall('"rhx_gis":"([0-9a-z]+)"', html, re.S)[0]
    print(GIS_rhx_gis)
    
    for item in items:
        if item.text().strip().startswith('window._sharedData'):
           
            js_data = json.loads(item.text()[21:-1], encoding='utf-8')
            #print(js_data)
            user_id = js_data['entry_data']['TagPage'][0]['graphql']['hashtag']['name']
            print(user_id)

            edges = js_data["entry_data"]["TagPage"][0]["graphql"]["hashtag"]["edge_hashtag_to_media"]["edges"]
            page_info = js_data["entry_data"]["TagPage"][0]["graphql"]["hashtag"]["edge_hashtag_to_media"]['page_info']
            cursor = page_info['end_cursor']
            print(cursor)
            flag = page_info['has_next_page']
            
            for edge in edges:
                try:
                    profile['id'] = edge['node']['owner']['id']
                except:
                    profile['id']
                    continue
                    
                try:
                    profile['text'] = edge['node']['edge_media_to_caption']['edges'][0]['node']['text']
                except:
                    profile['text'] = "None"
                    continue
                    
                try:
                    profile['picLink'] = edge['node']['display_url']
                except:
                    profile['picLink'] = "None"
                    continue
                    
                try:
                    profile['count'] = edge['node']['edge_liked_by']['count']
                except:
                    profile['count'] = "None"
                    continue
                    
                try:
                    profile['comment'] = edge['node']['edge_media_to_comment']['count']
                except:
                    profile['comment'] = "None"
                    continue
                    
                data.append(profile)
                profile = {}
                
    
    while flag:
        
        queryVariables = '{"tag_name":"' + user_id + '","show_ranked":"false","first":12,"after":"' +cursor+ '"}'
        #print(queryVariables)
        url_1 = uri.format(user_id=queryVariables)
        print(url_1)
        headers['X-Instagram-GIS'] = hashStr(GIS_rhx_gis + ":" + queryVariables)
        print(headers['X-Instagram-GIS'])
                
        js_data = get_json(url_1,headers)
        infos = js_data['data']["hashtag"]["edge_hashtag_to_ranked_media"]["edges"]
        cursor = js_data['data']["hashtag"]["edge_hashtag_to_ranked_media"]['page_info']['end_cursor']
        print(cursor)
        flag = js_data['data']["hashtag"]["edge_hashtag_to_ranked_media"]['page_info']['has_next_page']
            
        for info in infos:
            
            try:
                profile['id'] = info ['node']['owner']['id']
                print(profile['id'])
            except:
                profile['id'] = "None"
                continue
                
            try:
                profile['text'] = info ['node']['edge_media_to_caption']['edges'][0]['node']['text']
            except:
                profile['text'] = "None"
                continue
                
            try:
                profile['picLink'] = info ['node']['display_url']
            except:
                profile['picLink'] = "None"
                continue
                
            try:
                profile['count'] = info ['node']['edge_liked_by']['count']
            except:
                profile['count'] = "None"
                continue
                
            try:
                profile['comment'] = info ['node']['edge_media_to_comment']['count']
            except:
                profile['comment'] = "None"
                continue
                
            data.append(profile)
            profile = {}
        print(j)
        j=j+1
        time.sleep(3)
    
            
    return profile

In [31]:
html = get_urls(url)
crawl(html)

ead3f12ce6de3a2d8b5bb97a14821f27
台南
QVFCSTBBOVhKeXVSWnJFZ2x0dUtqQjJHbVhLazVPMTRQV2ZQUlcxeDBPeGNjcmRUR0hsUTdBT0VMQmNTb3VhRWNsNlBDdkNiNUFVZzhucnlDNlZIbTFjNg==
https://www.instagram.com/graphql/query/?query_hash=f92f56d47dc7a55b606908374b43a314&variables={"tag_name":"台南","show_ranked":"false","first":12,"after":"QVFCSTBBOVhKeXVSWnJFZ2x0dUtqQjJHbVhLazVPMTRQV2ZQUlcxeDBPeGNjcmRUR0hsUTdBT0VMQmNTb3VhRWNsNlBDdkNiNUFVZzhucnlDNlZIbTFjNg=="}
3cdd367531a24bb5a3e60d77a27835f8
QVFCSTBBOVhKeXVSWnJFZ2x0dUtqQjJHbVhLazVPMTRQV2ZQUlcxeDBPeGNjcmRUR0hsUTdBT0VMQmNTb3VhRWNsNlBDdkNiNUFVZzhucnlDNlZIbTFjNg==
3108664643
222588971
11878128
1646219001
761854092
10493079851
3108664643
257490566
434240411
4076816935
404972011
1407206584
0
https://www.instagram.com/graphql/query/?query_hash=f92f56d47dc7a55b606908374b43a314&variables={"tag_name":"台南","show_ranked":"false","first":12,"after":"QVFCSTBBOVhKeXVSWnJFZ2x0dUtqQjJHbVhLazVPMTRQV2ZQUlcxeDBPeGNjcmRUR0hsUTdBT0VMQmNTb3VhRWNsNlBDdkNiNUFVZzhucnlDNlZIbTFjNg=="}
3cdd367

KeyboardInterrupt: 

In [27]:
j=0
for i in data:
    print(j)
    print('id:'+i['id']+'\n')
    print('picturelink:'+'\n'+i['picLink']+'\n')
    print('text:'+'\n'+i['text']+'\n')
    print('like:')
    print(i['count'])
    print('comment:')
    print(i['comment'])
    print("===========================================================================================")
    print("\n")
    j=j+1
    
        

0
id:288894079

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/d3a6b6d5cfeea6cd3dd645481384b93d/5D2535E1/t51.2885-15/e35/51287132_407281006508202_2085809177786313658_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
被廟內的起炮
震撼到手一直抖
這只是全台最猛
高空煙火
前的小菜一碟
就知道為什麼
每年都吸引了全台的人
來這的原因
沒體驗過的人都差點
被起砲前的開幕騙了

謝謝 @wu.tony922 卡了中殿王位
#台南
#台南土城正統鹿耳門聖母廟 
#鹿耳門
#鹿耳門聖母廟 
#起砲
#國際高空煙火
#煙火
#元宵
#新年快樂

like:
0
comment:
0


1
id:2012351207

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/2e5725a474071eb68ec6d9edb8774b02/5CF77D24/t51.2885-15/e35/50867101_370728823749435_3722573257752751965_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
久違了 覺得該培養另外ㄧ種心境 遲來的 7年5個月❤️
簡單就是幸福 #奇美博物館 #台南

like:
0
comment:
0


2
id:462018691

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/27960ce553eefb1d67bc182030b00a9c/5CF0BF47/t51.2885-15/e35/51062487_2231625336897514_5436542709834650237_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
台南小西門B1💈
#thegoodforitbarberclub 
#goodforit
#tainan
#mitsukoshi 


text:
小腳怪
-
喬很久🤣🤣
-
#instaphoto #2019scootertrip #smallfoot #monster #scootertrip#lighthouse #燈塔 #trip #🛵🚶🏻‍♀️ #nature#naturephotography #極西點 #bighands #instagram#igtaiwan#ig_taiwan#instagood #outdoors#instaphotography #natgeotravel#iseetaiwan#amazingtaiwan #bcctravel #yellowjacketridder #photography #depthsofearth#funny#landscape #igphoto #台南
-
20190205/大年初一

like:
85
comment:
4


187
id:3627212665

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/0096e7f6cf68d6ae1758c97621c82255/5CFB7442/t51.2885-15/e35/49815620_812422989096244_8697012100650252529_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
老味道🗿 1989 BMW E30 325i🚙

#bmw #bmwm3 #bmwe30 #e30coupe #oldschoolbmw #oldschool #classicbmw #classiccar #classic #bimmerpost #bimmerboy #bimmer #stance #stanceworks #vintage #vintagecar #e30 #low #80s #8classic #follow #followme #photooftheday #awesome #goodmorning #car #photo #台南 #holiday #followforfollowback

like:
1049
comment:
82


188
id:1366801952

picturelink:
https://instagr


text:
_
驚險趕場
七點還在路上塞
感謝隊友卡位支援
欠個煙火人情
所以最後發圖來還
_
鹿耳門的煙火
勝過跨年任何煙火
-#huntgramtaiwan#taipei#photoofthesay#taiwan#photographer#photographylovers#vsco#vscotaiwan#iseetaiwan#discover_taiwan#igerstaiwan#instagram#ig_taiwan#exploretaiwan#bravotaiwan#台南#燈會 #元宵節 #鹿耳門聖母廟 #鹿耳門天后宮 #煙火 #土城

like:
303
comment:
0


287
id:40643184

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/e7743d5c340a96a76f63d22d71f7b7f7/5CE8B6AD/t51.2885-15/e35/50995304_2245622999021934_1425253503874994731_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
要出2了喔喔喔喔喔喔喔喔
我豪興奮ㄚㄚㄚㄚ
Elsa公主穿新衣•ू(ᵒ̴̶̷ωᵒ̴̶̷*•ू))੭ु⁾
（預告的造型）
_﻿﻿﻿﻿﻿﻿﻿
﻿﻿﻿﻿﻿﻿﻿﻿
﻿﻿﻿#frozen #frozen2 #2019 #CG 
#elsa #design #animate #movie #sketch #illustration #black #color ﻿﻿﻿#medibangpaint #ink #instaart #painting #ipadpro ﻿ ﻿#tattooartist ﻿ #instagood ﻿﻿﻿#drawing ﻿﻿﻿﻿﻿﻿﻿﻿
#電繪 #畫畫 #台南﻿ ﻿﻿﻿#tainan ﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿
_﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿ ﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿
﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿
刺青預約私訊﻿﻿LINE: @qlu2283k

like:
53
comment:
2


288
id:568734802

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/86a90c1769ff109bb5d6bfa9d79d75b9/5CE7F219/t51.2885-15/e35/51608635_2135893473136329_5382407454541966330_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
新開館！來培養所剩無幾的藝術氣息。看能不能透過藝術家的眼睛，增加一點生命厚度XD #台南 #台南美術館 #tainantravel #artmuseum

like:
4
comment:
0


437
id:9505584804

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/c9df3fb192c46b2dbc677a1fb854b16e/5CE91178/t51.2885-15/e35/52574119_2096144877141894_2418364579463395801_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
#七股鹽山
位於台南七股
為七股鹽場末代的曬鹽堆疊的人造山
近幾年都會搭上一隻吉祥物！
白茫茫的山搭上藍天
彷彿到了雪地一樣😍
#台南景點 #台南旅遊 #七股鹽山 #七股景點 #七股 #台南 #鹽山 ##tainan #taiwan #tainantravel

like:
12
comment:
0


438
id:7798891006

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/30e0fe72412e1cc171c98591b1f8579b/5CE96D8C/t51.2885-15/e35/51725729_325837644724960_23512564397301693_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
台南，201901
.
📷canon ae1
🎞Kodak
✏️無
.
#底片攝影 #ae1 #台南 #德記洋行 #kodak #老房子

like:



picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/7d3ab1945f3ac0ca5d370f5dbe64b7db/5CE4A3E0/t51.2885-15/e35/51740301_296411691041872_8076582874180249817_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
鹿耳門國際高空煙火Part 1.🎇
-
一整個以為第一階段已經過了馬上跑去墊內，結果傻眼...
明年有要拍的話只拍外圍就好呵呵..
-
#臺灣 #台南 #insta派
#Taiwan #Tainan
#amazingtaiwan #iseetaiwan #vscotaiwan #taiwan1 #ig_taiwan #bpintaiwan #bns_taiwan #bns_city #discover_taiwan #eventtaiwan #bravotaiwan #lovetaiwan #igerstaiwan #viewtaiwan #vscocam #ig_color #citystreets #travelyam #landscapes #scenery #sonyalpha6000

like:
844
comment:
32


637
id:10493079851

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/d502e5e45ef9131360750b2f8e4a9168/5D27742F/t51.2885-15/e35/51978528_616279475481032_842613777896616379_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
你是否喜歡這樣的我呢?#外拍#氣質#孔廟#台南#沈思#人像寫真#拍照#正妹#妹子#長腿#美女#美腿#外拍麻豆#外拍模特兒#taiwangirl#表特#photooftheday#beauty

like:
1802
comment:
71


638
id:257490566

picturelink:
https://instagram.fkhh1-1.fn

id:1381831586

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/3bfc3ace4a70c2266e4faf28a24c5bcd/5D0305A0/t51.2885-15/e35/51703168_504707953388967_9196040438703055399_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
|0216|
開學前最後一個假日🏮
#台南 #台南景點 #普濟殿 #燈籠

like:
111
comment:
0


837
id:1494957918

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/cc23d35ac4d8de4ee12efb795036431b/5CF98B6A/t51.2885-15/e35/51861366_547548299062987_5747615395699960487_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
#有批牛肉好便宜 其實就是牛腱肉捲餅.餅皮薄咬起來QQ在搭配新鮮蔬菜跟特製醬料.整體口感很完美😋
.
⏰ㄧ~日15:00~20:00
🏠台南市中西區國華街二段159號
.
#台南#台南美食#中西區#國華街#國華街美食#排隊美食#吃貨#手機食先#好吃#有批牛肉好便宜

like:
380
comment:
12


838
id:35977982

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/683b012643f419cae1e3a8c2a269fd5f/5CE9EB3F/t51.2885-15/e35/p1080x1080/51548801_408496846571416_6840454321747531113_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
開心今年有你陪我一起看煙火❤️
————————————
現場真的好震撼
謝謝 @hzy.s @jonathan_05tw 
還有 @shiuan_0629 奶奶的開心農場

986
id:503405945

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/1d552b8ab765ef936704fe77863e083f/5CEDA33B/t51.2885-15/e35/50875619_2129570090460813_8860933842540945112_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
吃吃古早味圓仔湯

#台灣 #Taiwan #台南 #Tainan #台灣味 #Taiwanfood #スウィーツ #Sweets #shaveice #かき氷 #方吉 #菇菇 #nameko #なめこ #嚕嚕米 #muumi #ムーミン #Moomin #moomintroll

like:
6
comment:
0


987
id:40609207

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/1ffd749787394d4747d75ab7e6c326c1/5CF8C2B1/t51.2885-15/e35/50801933_290596588294760_1691984640419645598_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
再度光臨，這次選了北海道味噌炙燒拉麵，叉燒很厚給了三片，好喝到可以讓我喝光光。湯頭鹹度，麵條軟硬粗細都可以客製調整，價位不到200。加麵不加價，超好吃👍Hokaido miso bbq pork ramen. Nice japanese 🇯🇵 ramen restaurant at a resonable price, the saltness of soup, thickness and softness of noodle are all customized per your requirement, free of charge for extra noodle. so tasty 👍

#Taiwan #Tainan #Tainanfood #foodie #tainanfoodie #igfood #igfoodie #instafoo

1136
id:3027148859

picturelink:
https://instagram.fkhh1-1.fna.fbcdn.net/vp/554f6972fed900eee6dce1a64a479eee/5D266C65/t51.2885-15/e35/51229606_2165556063509894_5588928879533172884_n.jpg?_nc_ht=instagram.fkhh1-1.fna.fbcdn.net

text:
食1️⃣3️⃣1️⃣7️⃣：#地家食堂#抽獎
小巷內超隱密的藝術聚落🏡
老屋改造採光很棒 裡面的空間感也很舒適
主打南部在地小農食材的健康早午餐
快樂農夫沙拉250元吃到飽🥳🥳🥳
以及媽媽味的古早味肉燥鍋！超推👍🏼
還可以來這裡買菜唷～都市農場的感覺🥕
-
🔺歐姆驚喜蛋套餐 $220
🔺泰式酸辣火龍果冷麵套餐 $250
🔺日式味自慢自成藝閣套餐 $300
🔺招牌媽媽肉燥鍋 $320
-
#招牌媽媽肉燥鍋 超特別的對吧(⁎⁍̴̛ᴗ⁍̴̛⁎)
原來肉燥飯不只能用吃的 變成火鍋也可以
一鍋3吃👉🏼滷鍋、湯鍋、雜炊～
肉燥配上醬瓜古早味十足🤤吃起來也完全不死鹹剛剛好，我個人很喜歡這鍋😛
-
#泰式酸辣火龍果冷麵
火龍果製作成的麵條 超特別！
口感軟Q有嚼勁 配上微辣的泰式酸辣醬好唰嘴！也有日式風味套餐～
-
#快閃抽獎 抽3份「招牌媽媽肉燥鍋」
🎉使用期限至2019年3月底
1）按此篇❤️ 追蹤 @kelly_foodlife
2）tag朋友+留言：一起吃肉燥鍋
-
【快閃抽獎於 2/18 22:00結束，三日內公布】
可以重複留言但需標註不同朋友，不要標陌生人避免造成他人困擾😊謝謝配合🙇🏻‍♀️
-
☀️每人低消200元
📌台南美食👉🏼#Kelly食台南
📮台南市中西區民權路二段64巷11號
⏰11:00-20:00(週一二公休)
☎️06-2209866
-
FB/痞客邦🔍台南女孩凱莉吃透透
#foodie#foodstagram#foodphotography#vscofood#instafood#popdaily#popyummy#yummyday#台南美食#台南咖啡廳#台南早午餐#台南

like:
2123
comment:
275


1137
id:3415506111

picturelink:
ht

In [14]:
def file(data):
    with open("instaCrawler_#台南",mode="w",encoding="utf-8")as file:
        json_string=json.dumps(data)
        file.write(json_string)

In [29]:
file(data)

In [ ]:
https://www.instagram.com/graphql/query/?query_hash=f92f56d47dc7a55b606908374b43a314&variables={"tag_name":"奇美博物館","show_ranked":false,"first":6,"after":"QVFBV0t0aXc0R0pMQmN5a1RzdFpfYmcwcEJGZC16SnN5N0V3VUUwanIwNnZnT180dENldDMxNmhmYWg1XzdJRmlkWUdfbV84MzZkSF9lcF9hUk5ucmtqdQ=="}

In [ ]:
https://www.instagram.com/graphql/query/?query_hash=e6a78c2942f1370ea50e04c9a45ebc44&variables={"id":"292490918","first":12,"after":"QVFBb0hNczRERDk5aUg4MDJxa19kMDNxWUtLMFlpR0U2QUVCcHhXVzdMc3Y2VnhHNGJObm9RVmZJeWJjREpDdEN4UEFIZW9KVnlJMjQ5U1VkZmJNbUpOdw=="}